In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

2024-12-03 17:10:57.995537: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 17:10:58.015633: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733242258.048204  553145 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733242258.056100  553145 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 17:10:58.085491: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Generar datos para el módulo de llevadas
def generate_carry_data():
    x_data = []
    y_data = []
    for a in range(10):  # Primer número
        for b in range(10):  # Segundo número
            for c in range(2):
                x_data.append([a, b, c])  # Entrada
                y_data.append(1 if (a + b + c) >= 10 else 0)
    return np.array(x_data), np.array(y_data)

# Generar datos
x, y = generate_carry_data()

# Convertir etiquetas a one-hot (aunque solo haya 2 clases, 0 y 1)
y_one_hot = to_categorical(y, num_classes=2)

# Dividir en conjuntos de entrenamiento y validación (80% entrenamiento, 20% validación)
train_size = int(len(x) * 0.8)  # 80% de datos para entrenamiento
x_train, y_train = x[:train_size], y_one_hot[:train_size]
x_val, y_val = x, y_one_hot  # 20% para validación

# Asegurar formas consistentes para las entradas
x_train = np.expand_dims(x_train, axis=1)  # Agrega una dimensión para 'timesteps'
x_val = np.expand_dims(x_val, axis=1)      # Haz lo mismo con los datos de validación

# Construir el modelo
model = Sequential([
    LSTM(16, input_shape=(1, 3), return_sequences=True),
    LSTM(32, return_sequences=False),
    Dense(2, activation='softmax')  # Salida para 2 clases: 0 (sin llevada) o 1 (con llevada)
])

# Callback personalizado para detener cuando todas las combinaciones sean correctas
class StopWhenPerfectCallback(EarlyStopping):
    def __init__(self, val_data, **kwargs):
        super().__init__(**kwargs)
        self.val_data = val_data  # Pasamos explícitamente los datos de validación
    
    def on_epoch_end(self, epoch, logs=None):
        # Evaluar el rendimiento sobre todo el conjunto de validación
        val_predictions = self.model.predict(self.val_data[0])
        val_pred_labels = np.argmax(val_predictions, axis=1)
        val_true_labels = np.argmax(self.val_data[1], axis=1)
        
        # Verificar si todas las combinaciones son correctas
        correct_predictions = np.sum(val_pred_labels == val_true_labels)
        total_predictions = len(val_true_labels)
        
        print(f'Evaluación de validación: {correct_predictions}/{total_predictions} correctas')

        # Si todas las combinaciones son correctas, detener el entrenamiento
        if correct_predictions == total_predictions:
            print("¡Todas las combinaciones han sido aprendidas correctamente! Deteniendo entrenamiento.")
            self.model.stop_training = True

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

# Crear una instancia del callback con los datos de validación
stop_callback = StopWhenPerfectCallback(val_data=(x_val, y_val), patience=500)

# Entrenar el modelo con el callback personalizado
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=500,
    batch_size=1,
    callbacks=[stop_callback]
)

# Evaluación final
loss, accuracy = model.evaluate(x_val, y_val)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# Predicciones
predictions = model.predict(x_val)
predicted_numbers = np.argmax(predictions, axis=1)
real_numbers = np.argmax(y_val, axis=1)  # Etiquetas reales

print("Predicciones (probabilidades):", predictions[:5])
print("Predicciones (números):", predicted_numbers[:5])
print("Etiquetas reales:", real_numbers[:5])

# Mostrar las predicciones de validación y sus etiquetas
print("\nEvaluación de validación final:")
correct_predictions = np.sum(predicted_numbers == real_numbers)
total_predictions = len(real_numbers)
print(f'Predicciones correctas: {correct_predictions}/{total_predictions}')

# Guardar el modelo
model.save('dec_carry_module.keras')

Epoch 1/500


W0000 00:00:1733242261.994439  553145 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/samuel_lozano/CURRICULUM_LEARNING/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step ep - accuracy: 0.7065 - loss: 0.6
Evaluación de validación: 176/200 correctas
160/160 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7079 - loss: 0.6167 - val_accuracy: 0.8800 - val_loss: 0.2949
Epoch 2/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step tep - accuracy: 0.9382 - loss: 0.184
Evaluación de validación: 176/200 correctas
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9379 - loss: 0.1845 - val_accuracy: 0.8800 - val_loss: 0.2252
Epoch 3/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step tep - accuracy: 0.8720 - loss: 0.245
Evaluación de validación: 186/200 correctas
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8743 - loss: 0.2397 - val_accuracy: 0.9300 - val_loss: 0.1559
Epoch 4/500
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step tep - accuracy: 0.9185 - loss: 0.157
Evaluación de validación: 191/200 correctas
160/160 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9180 - loss: 0.1589 - val_accuracy: 0.9550 - val_loss: 0.1264
Epoch 5/500
7/7 ━━━━━━━━━